In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from utility import *

In [2]:
homedir = get_homedir(verbose=True)

c:\Users\juhn3\Documents\GitHub\SKTW


## Geography

In [3]:
# Geographic and population centers for each county in US.
raw_county_centers = pd.read_csv(f"{homedir}/data/us/geolocation/county_centers.csv")
raw_county_centers['fips'] = raw_county_centers['fips'].apply(correct_FIPS)
raw_county_centers.head()

fips     clon00     clat00     clon10     clat10    pclon00    pclat00  \
0  01001 -86.577176  32.523283 -86.644490  32.536382 -86.501832  32.500323   
1  01003 -87.748260  30.592781 -87.746067  30.659218 -87.760540  30.565383   
2  01005 -85.331312  31.856515 -85.405456  31.870670 -85.306746  31.847869   
3  01007 -87.123243  33.040054 -87.127148  33.015893 -87.127019  33.025947   
4  01009 -86.554768  33.978461 -86.567246  33.977448 -86.582617  33.962601   

     pclon10    pclat10  
0 -86.494165  32.500389  
1 -87.762381  30.548923  
2 -85.310038  31.844036  
3 -87.127659  33.030921  
4 -86.591491  33.955243

In [4]:
len(raw_county_centers)

3147

In [5]:
fips_to_county_name = pd.read_csv(f"{homedir}/data/us/processing_data/fips_key.csv", encoding='latin-1')
fips_to_county_name['FIPS'] = fips_to_county_name['FIPS'].apply(correct_FIPS)
fips_to_county_name.head()

FIPS  MSA/PMSA NECMA  ST          COUNTY
0  01001            5240  AL  Autauga County
1  01003            5160  AL  Baldwin County
2  01005               0  AL  Barbour County
3  01007               0  AL     Bibb County
4  01009            1000  AL   Blount County

In [6]:
len(fips_to_county_name)

3246

In [7]:
set(raw_county_centers["fips"]).difference(set(fips_to_county_name["FIPS"]))

{'02105', '02195', '02198', '02230', '02275', '08014'}

In [8]:
len(set(fips_to_county_name["FIPS"]).difference(set(raw_county_centers["fips"])))

82

## Submission format

In [9]:
df = pd.read_csv(f"{homedir}/sample_submission.csv", index_col=0)

In [10]:
df.head()

10   20   30   40   50   60   70   80   90
id                                                           
2020-04-01-10001  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2020-04-01-10003  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2020-04-01-10005  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2020-04-01-1001   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2020-04-01-1003   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [11]:
columnname = [f"{10*i}" for i in range(1,10)]
a = pd.DataFrame(index=df.index, columns=columnname)

In [12]:
len(a)//91 # Num. of FIPS'

3223

In [13]:
a_firstday=a.iloc[0:3223].index.to_numpy()
FIPS = []
for i in range(len(a_firstday)):
    FIPS.append(correct_FIPS(a_firstday[i][11:]))
FIPS[-1]

'09015'

In [14]:
print(set(fips_to_county_name["FIPS"])==set(FIPS))
print(len(set(FIPS))) # No duplications in FIPS

True
3223


In [15]:
dup_FIPS = set([x for x in fips_to_county_name["FIPS"] if fips_to_county_name["FIPS"].to_numpy().tolist().count(x) > 1])
for dup in dup_FIPS:
    print(fips_to_county_name[fips_to_county_name["FIPS"]==dup]["COUNTY"].to_numpy())

['De Kalb County' 'DeKalb County']
['DeKalb County' 'De Kalb County']
['De Baca County' 'DeBaca County']
['Doí±a Ana County' 'Dona Ana County']
['Hoonah-Angoon Census Area' 'Skagway Municipality'
 'Skagway/Hoonah/Angoon Census Area']
['Petersburg Borough' 'Wrangell City and Borough'
 'Wrangell-Petersburg Census Area']
['Laporte County' 'La Porte County']
['Prince of Wales-Hyder Census Area'
 'Prince of Wales-Outer Ketchikan Census Area']
['McKean County' 'Mc Kean County']
['DeKalb County' 'De Kalb County']
['Sitka City and Borough' 'Sitka Borough']
['Anchorage Municipality' 'Anchorage Borough']
['Juneau City and Borough' 'Juneau Borough']
['Lasalle County' 'La Salle County']
['Dupage County' 'Du Page County']
['Denali Borough' 'Denali Census Area' 'Denali Census Area']
['De Kalb County' 'DeKalb County']
['Lasalle Parish' 'La Salle Parish']
['Lasalle County' 'La Salle County']
['Desoto County' 'De Soto County']


## USAfacts dataset

In [16]:
confirmed_cases = pd.read_csv(f"{homedir}/data/us/covid/confirmed_cases.csv", usecols=["countyFIPS", "County Name"])
confirmed_cases.head()

countyFIPS            County Name
0           0  Statewide Unallocated
1        1001         Autauga County
2        1003         Baldwin County
3        1005         Barbour County
4        1007            Bibb County

In [17]:
deaths = pd.read_csv(f"{homedir}/data/us/covid/deaths.csv", usecols=["countyFIPS", "County Name"])
deaths.head()

countyFIPS            County Name
0           0  Statewide Unallocated
1        1001         Autauga County
2        1003         Baldwin County
3        1005         Barbour County
4        1007            Bibb County

In [18]:
len(deaths)

3195

In [19]:
(confirmed_cases==deaths).sum()

countyFIPS     3195
County Name    3179
dtype: int64

In [20]:
confirmed_cases["County Name"][confirmed_cases["County Name"]!=deaths["County Name"]]

259     Broomfield County and City
1375          Lac Qui Parle County
1835               Dona Ana County
2925               Matthews County
2964               Alexandria City
2967          Charlottesville City
2968               Chesapeake City
2971                 Danville City
2976           Fredericksburg City
2979             Harrisonburg City
2983                 Manassas City
2987                  Norfolk City
2991               Portsmouth City
2993                 Richmond City
2997                  Suffolk City
2998           Virginia Beach City
Name: County Name, dtype: object

In [21]:
deaths["County Name"][confirmed_cases["County Name"]!=deaths["County Name"]] # Mostly capitalization issues, with 2 exceptions(Broomfield, Dona Ana)

259        Broomfield County
1375    Lac qui Parle County
1835       DoÔøΩa Ana County
2925          Mathews County
2964         Alexandria city
2967    Charlottesville city
2968         Chesapeake city
2971           Danville city
2976     Fredericksburg city
2979       Harrisonburg city
2983           Manassas city
2987            Norfolk city
2991         Portsmouth city
2993           Richmond city
2997            Suffolk city
2998     Virginia Beach city
Name: County Name, dtype: object

In [22]:
deaths["countyFIPS"].nsmallest(60).to_numpy() # 0 for statewide unallocated, 1 for NYC unallocated

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    1, 1001, 1003, 1005, 1007,
       1009, 1011, 1013, 1015, 1017], dtype=int64)

In [23]:
FIPS_USA = deaths["countyFIPS"].apply(correct_FIPS).to_numpy()

In [24]:
s = set(FIPS_USA)
s.remove('0')
s.remove('1')
# s.remove('06000') # Grand Princess Cruise
min(s)

'01001'

In [25]:
s.difference(set(FIPS))

{'02105',
 '02158',
 '02195',
 '02198',
 '02230',
 '02275',
 '06000',
 '08014',
 '46102'}

In [26]:
for fips in s.difference(set(FIPS)):
    print(fips, confirmed_cases["County Name"].loc[confirmed_cases["countyFIPS"].apply(correct_FIPS)==fips].item())

02105 Hoonah-Angoon Census Area
08014 Broomfield County and City
02195 Petersburg Census Area
02230 Skagway Municipality
46102 Oglala Lakota County
02275 Wrangell City and Borough
02198 Prince of Wales-Hyder Census Area
06000 Grand Princess Cruise Ship
02158 Kusilvak Census Area


In [27]:
len(set(FIPS).difference(s))

88

## NYT dataset

In [28]:
NYT = pd.read_csv(f"{homedir}/data/us/covid/nyt_us_counties.csv")
NYT.head()

date     county       state     fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061.0      1       0
1  2020-01-22  Snohomish  Washington  53061.0      1       0
2  2020-01-23  Snohomish  Washington  53061.0      1       0
3  2020-01-24       Cook    Illinois  17031.0      1       0
4  2020-01-24  Snohomish  Washington  53061.0      1       0

In [29]:
NYT[["county", "state"]][NYT["fips"].isna()].drop_duplicates()

county                     state
416    New York City                  New York
418          Unknown              Rhode Island
1511         Unknown                New Jersey
1858         Unknown               Puerto Rico
2267         Unknown            Virgin Islands
2422         Unknown                      Guam
2929         Unknown                     Maine
2950         Unknown             Massachusetts
4003         Unknown                 Louisiana
4680         Unknown                  Kentucky
4828         Unknown                    Nevada
5038         Unknown                 Tennessee
5223         Unknown                  Arkansas
5385         Unknown                   Georgia
5641     Kansas City                  Missouri
5647         Unknown                  Missouri
6570         Unknown                 Minnesota
7215         Unknown                California
7246         Unknown                  Colorado
7304         Unknown                   Florida
8529         Unknown                    Hawaii
8564         Unknown                  Illinois
9359         Unknown                   Vermont
12759        Unknown                     Idaho
13116        Unknown                  Michigan
13531        Unknown                      Ohio
13860        Unknown                      Utah
14946        Unknown               Mississippi
17074        Unknown  Northern Mariana Islands
18815        Unknown                  Nebraska
19952        Unknown               Connecticut
21353        Unknown              South Dakota
22405        Unknown                   Indiana
26267        Unknown                    Alaska
28574        Unknown                   Arizona
29850        Unknown                New Mexico
29904        Unknown                  New York
30237        Unknown              Pennsylvania
30694        Unknown                  Virginia
32193        Unknown             New Hampshire
38064        Unknown                Washington
46165        Unknown                  Delaware
46739        Unknown                    Kansas
47579        Unknown              North Dakota

In [30]:
NYT_notna = NYT[NYT["fips"].notna()]
NYT_notna["fips"] = NYT_notna["fips"].apply(correct_FIPS)
NYT_notna.head()

date     county       state   fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061      1       0
1  2020-01-22  Snohomish  Washington  53061      1       0
2  2020-01-23  Snohomish  Washington  53061      1       0
3  2020-01-24       Cook    Illinois  17031      1       0
4  2020-01-24  Snohomish  Washington  53061      1       0

In [31]:
len(set(NYT_notna["fips"]))

2667

In [32]:
set(NYT_notna["fips"]).difference(set(FIPS))

{'02195', '02198', '08014', '46102'}

In [33]:
for fips in set(NYT_notna["fips"]).difference(set(FIPS)):
    print(fips, NYT_notna["county"].loc[NYT_notna["fips"]==fips].iloc[0]) # 02: Alaska

02198 Prince of Wales-Hyder Census Area
08014 Broomfield
46102 Oglala Lakota
02195 Petersburg Borough


## Berkeley

In [34]:
b = pd.read_csv("../data/us/aggregate_berkeley.csv", index_col=0)
b.head()

PopulationEstimate2018  Population(Persons)2017  PopTotalMale2017  \
0                 55601.0                  55504.0           27007.0   
1                218022.0                 212628.0          103225.0   
2                 24881.0                  25270.0           13335.0   
3                 22400.0                  22668.0           12138.0   
4                 57840.0                  58013.0           28607.0   

   PopTotalFemale2017  FracMale2017  PopulationEstimate65+2017  \
0             28497.0      0.486578                     8392.0   
1            109403.0      0.485472                    42413.0   
2             11935.0      0.527701                     4757.0   
3             10530.0      0.535469                     3632.0   
4             29406.0      0.493114                    10351.0   

   PopulationDensityperSqMile2010  CensusPopulation2010  MedianAge2010  \
0                            91.8               54571.0           37.0   
1                           114.7              182265.0           41.1   
2                            31.0               27457.0           39.0   
3                            36.8               22915.0           37.8   
4                            88.9               57322.0           39.0   

   MedianAge,Male2010  ...  3-YrMortalityAge5-14Years2015-17  \
0                35.9  ...                               0.0   
1                40.1  ...                               0.0   
2                37.2  ...                               0.0   
3                36.5  ...                               0.0   
4                38.0  ...                               0.0   

   3-YrMortalityAge15-24Years2015-17  3-YrMortalityAge25-34Years2015-17  \
0                                0.0                               10.0   
1                               23.0                               30.0   
2                                0.0                                0.0   
3                                0.0                                0.0   
4                               10.0                               13.0   

   3-YrMortalityAge35-44Years2015-17  3-YrMortalityAge45-54Years2015-17  \
0                               20.0                               44.0   
1                               52.0                              125.0   
2                                0.0                               21.0   
3                                0.0                               24.0   
4                               25.0                               49.0   

   3-YrMortalityAge55-64Years2015-17  3-YrMortalityAge65-74Years2015-17  \
0                               73.0                              112.0   
1                              262.0                              434.0   
2                               44.0                               65.0   
3                               37.0                               62.0   
4                               94.0                              150.0   

   3-YrMortalityAge75-84Years2015-17  3-YrMortalityAge85+Years2015-17  \
0                              143.0                            109.0   
1                              546.0                            597.0   
2                               72.0                             68.0   
3                               56.0                             45.0   
4                              182.0                            142.0   

   mortality2015-17Estimated  
0                  33.194755  
1                 148.887537  
2                  18.483155  
3                  15.381366  
4                  48.499442  

[5 rows x 64 columns]

In [35]:
b.columns

Index(['PopulationEstimate2018', 'Population(Persons)2017', 'PopTotalMale2017',
       'PopTotalFemale2017', 'FracMale2017', 'PopulationEstimate65+2017',
       'PopulationDensityperSqMile2010', 'CensusPopulation2010',
       'MedianAge2010', 'MedianAge,Male2010', 'MedianAge,Female2010',
       '#EligibleforMedicare2018', 'MedicareEnrollment,AgedTot2017',
       '3-YrDiabetes2015-17', 'DiabetesPercentage', 'HeartDiseaseMortality',
       'StrokeMortality', 'Smokers_Percentage', '#FTEHospitalTotal2017',
       'TotalM.D.'s,TotNon-FedandFed2017', '#HospParticipatinginNetwork2017',
       '#Hospitals', '#ICU_beds', 'dem_to_rep_ratio', 'PopMale<52010',
       'PopFmle<52010', 'PopMale5-92010', 'PopFmle5-92010', 'PopMale10-142010',
       'PopFmle10-142010', 'PopMale15-192010', 'PopFmle15-192010',
       'PopMale20-242010', 'PopFmle20-242010', 'PopMale25-292010',
       'PopFmle25-292010', 'PopMale30-342010', 'PopFmle30-342010',
       'PopMale35-442010', 'PopFmle35-442010', 'PopMale45-5420